# 필요한 라이브러리 불러오기

In [313]:
import urllib.request
from bs4 import BeautifulSoup
from itertools import product
from pandas import DataFrame


필요한 라이브러리를 불러옵니다.

# 필요한 기능들 정의하기.

In [314]:
def table_to_2d(table_tag):
    rowspans = []  # track pending rowspans
    rows = table_tag.find_all('tr')

    # first scan, see how many columns we need
    colcount = 0
    for r, row in enumerate(rows):
        cells = row.find_all(['td', 'th'], recursive=False)
        # count columns (including spanned).
        # add active rowspans from preceding rows
        # we *ignore* the colspan value on the last cell, to prevent
        # creating 'phantom' columns with no actual cells, only extended
        # colspans. This is achieved by hardcoding the last cell width as 1. 
        # a colspan of 0 means “fill until the end” but can really only apply
        # to the last cell; ignore it elsewhere. 
        colcount = max(
            colcount,
            sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
        # update rowspan bookkeeping; 0 is a span to the bottom. 
        rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
        rowspans = [s - 1 for s in rowspans if s > 1]

    # it doesn't matter if there are still rowspan numbers 'active'; no extra
    # rows to show in the table means the larger than 1 rowspan numbers in the
    # last table row are ignored.

    # build an empty matrix for all possible cells
    table = [[None] * colcount for row in rows]

    # fill matrix from row data
    rowspans = {}  # track pending rowspans, column number mapping to count
    for row, row_elem in enumerate(rows):
        span_offset = 0  # how many columns are skipped due to row and colspans 
        for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
            # adjust for preceding row and colspans
            col += span_offset
            while rowspans.get(col, 0):
                span_offset += 1
                col += 1

            # fill table data
            rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
            colspan = int(cell.get('colspan', 1)) or colcount - col
            # next column is offset by the colspan
            span_offset += colspan - 1
            value = cell.get_text()
            for drow, dcol in product(range(rowspan), range(colspan)):
                try:
                    table[row + drow][col + dcol] = value
                    rowspans[col + dcol] = rowspan
                except IndexError:
                    # rowspan or colspan outside the confines of the table
                    pass

        # update rowspan bookkeeping
        rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}

    return table

가장 핵심이 되는 기능입니다. table tag를 넣으면, list로 바꿔줍니다.

In [315]:
def deleteEnter(tableList):
    newTableList = []
    for itemList in tableList: # itemlist는 각 row
        newItems = []
        for item in itemList: # 각 row 안의 item
            x = item.split('\n') # 각 스트링 앞뒤에 \n이 붙어 있습니다. 제거합니다. 그 결과로 각 아이템 앞뒤에 ''가 붙습니다.
            x_ = x[:] 
            for y in x_:
                if(len(y)==0): 
                    x.remove(y) # ''를 제거합니다.
            newItems.append('\n'.join(x)) # 셀 안에서 앤터를 쳐야하는 데이터들이 있었기 때문에 이를 반영했습니다.
        newTableList.append(newItems)
    return newTableList

데이터 안의 \n을 제거합니다.

# 데이터 불러오기

In [316]:
page = urllib.request.urlopen('https://news.seoul.go.kr/html/27/525381/')
html_string = str(page.read().decode("utf8"))
print(html_string)

























<p>▴손님 예약제 운영</p>
































<p>▴손님 음료 제공 금지</p>


</td>


</tr>
































<tr>


<td rowspan="2">


<p><b>상점</b><b>·</b><b>마트</b><b>·</b></p>
































<p><b>백화점</b></p>


</td>
































<td class="td-left" rowspan="2">


<p>▴마스크 착용, 환기･소독</p>


</td>
































<td class="td-left" style="border-left: 2px solid blue; border-right: 2px solid blue;">


<p>변동없음</p>


</td>


</tr>
































<tr>


<td class="td-left" style="border-left: 2px solid blue; border-right: 2px solid blue; background: #f2f2f2; color: #315fa7;">


<p><b>&lt; </b><b>서울형 강화 조치 </b><b>&gt;</b></p>
































<p>▴코로나19 감염 의심 종사자 관리</p>
































<p>- 발열체크, 호흡기 증상･발열시 출근 중단 및 즉시 퇴근조치 등</p>
































<p>▴손씻기 등 안내문 게시 및 손소독제 비치</p>
































<p>▴이벤트성 행사, 시식･시음 코너 운영 금지</p>











테이블을 불러올 사이트 입니다. 서울시 코로나 관련 홈페이지이며, 이 중 임시선별검사소 내용을 꺼내올 것입니다.

In [317]:
soup = BeautifulSoup(html_string, 'lxml')
table = soup.find_all('table')[0] # Grab the first table

html안에서 테이블을 꺼내옵니다.

In [318]:
tableList = table_to_2d(table)
print(tableList)

[['연번', '자치구', '개소', '장 소', '개소일시', '운영시간', '비고'], ['\n계\n', '\n25\n', '\n52\n', '\n52개소\n', '\n52개소\n', '\n52개소\n', '\n52개소\n'], ['\n1\n', '\n종로\n', '\n2\n', '\n탑골공원 앞\n', '\n12.14~2.14\n', '\n평일(월~금) 10:00~17:00\n토요일 09:00~13:00\n', '\n평일 소독시간12~14\n'], ['\n1\n', '\n종로\n', '\n2\n', '\n종로구민회관 후문\n', '\n12.16~2.14\n', '\n평일(월~금) 10:00~17:00\n토요일 09:00~13:00\n', '\n평일 소독시간12~14\n'], ['\n2\n', '\n중구\n', '\n3\n', '\n서울역 광장\n', '\n12.14~2.14\n', '\n평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:00~13:00\n', '\n평일 소독시간12:30~13:30\n'], ['\n2\n', '\n중구\n', '\n3\n', '\n약수동 주민센터 앞\n', '\n12.15~2.14\n', '\n평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:00~13:00\n', '\n평일 소독시간12:30~13:30\n'], ['\n2\n', '\n중구\n', '\n3\n', '\n서울광장※ 주말 휴무\n', '\n12.17~2.14\n', '\n평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:00~13:00\n', '\n평일 소독시간12:30~13:30\n'], ['\n3\n', '\n용산\n', '\n2\n', '\n용산역 전면광장\n', '\n12.14~2.14\n', '\n평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:00~13:00\n', '\n평일 소독시간12~13\n'], ['\n3\n', '\n용

중간중간 \n(앤터)의 흔적이 있습니다. 지워주도록 합시다.

# Pandas를 이용해서 데이터 확인하기

In [319]:
newTableList = deleteEnter(tableList)
df = DataFrame (newTableList[2:],columns=newTableList[0])
df.head(52)

,연번,자치구,개소,장 소,개소일시,운영시간,비고
0,1,종로,2,탑골공원 앞,12.14~2.14,평일(월~금) 10:00~17:00\n토요일 09:00~13:00,평일 소독시간12~14
1,1,종로,2,종로구민회관 후문,12.16~2.14,평일(월~금) 10:00~17:00\n토요일 09:00~13:00,평일 소독시간12~14
2,2,중구,3,서울역 광장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
3,2,중구,3,약수동 주민센터 앞,12.15~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
4,2,중구,3,서울광장※ 주말 휴무,12.17~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
5,3,용산,2,용산역 전면광장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12~13
6,3,용산,2,한남동공영주차장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12~13
7,4,성동,2,성동구청(농구장),12.16~2.14,평일(월~금) 09:00~17:00\n※ 주말·공휴일 휴무,
8,4,성동,2,성수구두테마공원(성수역 4번출구 200m),12.16~2.10,평일(월~금) 09:00~17:00\n※ 주말·공휴일 휴무,
9,5,광진,2,중곡보건지소,12.15~2.14,평일(월~금) 09:00~17:00\n토요일 12:00~16:00\n일요일 12:0...,평일 소독시간13~14


개소 부분이 오해가 있을 거 같습니다. 지워주도록 합시다.

In [320]:
del df['개소']
df.head(52)

,연번,자치구,장 소,개소일시,운영시간,비고
0,1,종로,탑골공원 앞,12.14~2.14,평일(월~금) 10:00~17:00\n토요일 09:00~13:00,평일 소독시간12~14
1,1,종로,종로구민회관 후문,12.16~2.14,평일(월~금) 10:00~17:00\n토요일 09:00~13:00,평일 소독시간12~14
2,2,중구,서울역 광장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
3,2,중구,약수동 주민센터 앞,12.15~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
4,2,중구,서울광장※ 주말 휴무,12.17~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12:30~13:30
5,3,용산,용산역 전면광장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12~13
6,3,용산,한남동공영주차장,12.14~2.14,평일(월~금) 09:00~17:00\n토요일 09:00~13:00\n일요일 09:0...,평일 소독시간12~13
7,4,성동,성동구청(농구장),12.16~2.14,평일(월~금) 09:00~17:00\n※ 주말·공휴일 휴무,
8,4,성동,성수구두테마공원(성수역 4번출구 200m),12.16~2.10,평일(월~금) 09:00~17:00\n※ 주말·공휴일 휴무,
9,5,광진,중곡보건지소,12.15~2.14,평일(월~금) 09:00~17:00\n토요일 12:00~16:00\n일요일 12:0...,평일 소독시간13~14


참조 사이트 : https://www.codegrepper.com/code-examples/python/python+get+html+from+url <- url을 utf8로 decoding

https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/